In [1]:
import pandas as pd
import datetime
# To update the date from the output of this along with the new downloaed next week

In [2]:
today_str=str(datetime.datetime.now().date())
today_str=today_str[0:4]+today_str[5:7]+today_str[8:10]
recent_week_path="/home/jian/Projects/Saatva/Transaction_Data/2018_weekly_klipfolio/transactions_for_jubaplus-"+today_str+".csv"
recent_week_data=pd.read_csv(recent_week_path,dtype=str)
recent_week_data=recent_week_data[~pd.isnull(recent_week_data['Order ID'])]
recent_week_data['Date']=recent_week_data['Date'].apply(lambda x: datetime.datetime.strptime(x,"%d-%b-%Y").date())
recent_week_data['Time']=recent_week_data['Time'].apply(lambda x: datetime.datetime.strptime(x,"%H:%M:%S").time())
recent_week_data_1=recent_week_data[(recent_week_data['Date']>recent_week_data['Date'].min()) & (recent_week_data['Date']<=recent_week_data['Date'].min()+datetime.timedelta(days=30))]
recent_week_data_2=recent_week_data[(recent_week_data['Date']>recent_week_data_1['Date'].max()) & (recent_week_data['Date']<recent_week_data['Date'].max())]

In [3]:
zip_State_1=pd.read_excel("/home/jian/Projects/Saatva/Transaction_Data/Geo/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,dtype=str)
zip_State_1=zip_State_1.iloc[:,[0,6]]
zip_State_1.columns=['zip_cd','ST']
zip_State_1=zip_State_1.drop_duplicates(['zip_cd'])
zip_State_1.reset_index(inplace=True)
del zip_State_1['index']
zip_State_2=pd.read_csv("/home/jian/Projects/Saatva/Transaction_Data/Geo/us_postal_codes_online_download.csv",dtype=str)
zip_State_2=zip_State_2.iloc[:,[0,3]]
zip_State_2.columns=['zip_cd','ST']
zip_State_2=zip_State_2.drop_duplicates()
zip_State_2.reset_index(inplace=True)
zip_State_2['zip_cd']=zip_State_2['zip_cd'].apply(lambda x:x.zfill(5))
del zip_State_2['index']
zip_State_2=zip_State_2[~zip_State_2['zip_cd'].isin(zip_State_1['zip_cd'])]
zip_State_2=zip_State_2[~zip_State_2['zip_cd'].isin(zip_State_1['zip_cd'])]
zip_State=zip_State_1.append(zip_State_2)
zip_State.columns=['Shipping Zip','ST']

In [4]:
historical_order=pd.read_excel("/home/jian/Projects/Saatva/Transaction_Data/Order_data.xlsx",sheetname="Saatva",dtype=str)
historical_order=historical_order[['order_id','date','time','zipcode','product_revenue','online/phone?','brand']]
historical_order.columns=['Order ID', 'Date', 'Time', 'Shipping Zip', 'Product Revenue','Order Type', 'Brand']
historical_order['Date']=historical_order['Date'].apply(lambda x: datetime.datetime.strptime(x[0:10],"%Y-%m-%d").date())
historical_order['Time']=historical_order['Time'].apply(lambda x: datetime.datetime.strptime(x[0:10],"%H:%M:%S").time())

In [5]:
early_2018=pd.read_csv("/home/jian/Projects/Saatva/Transaction_Data/3 Brands_transactions_20180101-20180308.csv",dtype=str)
early_2018=early_2018[early_2018['Brand']=="Saatva"]

early_2018['Date']=early_2018['Date'].apply(lambda x: datetime.datetime.strptime(x,"%d-%b-%Y").date())
early_2018['Time']=early_2018['Time'].apply(lambda x: datetime.datetime.strptime(x,"%H:%M:%S").time())
early_2018=early_2018[~early_2018['Date'].isin(recent_week_data_1['Date'])]
early_2018.reset_index(inplace=True)
del early_2018['index']

In [6]:
data=historical_order.append(early_2018).append(recent_week_data_1).append(recent_week_data_2).reset_index()
del data['index']
data['Product Revenue']=data['Product Revenue'].astype(float)
data['Order Type']=data['Order Type'].replace("phone_order","phone")

In [7]:
data['Shipping Zip']=data['Shipping Zip'].apply(lambda x:x.zfill(5))

In [8]:
data=pd.merge(data,zip_State,on="Shipping Zip",how="left")
data['ST']=data['ST'].fillna("Non-US")

In [9]:
end_date=str(data['Date'].max())

In [10]:
data.to_csv("/home/jian/Projects/Saatva/Transaction_Data/output/Saatva_transaction_data_till_"+end_date+".csv",index=False)